<a href="https://colab.research.google.com/github/Chohyungun/Chohyungun/blob/Dais/%EB%8D%B0%EC%9D%B4%EC%BD%98_%EC%8A%A4%EC%BF%A8_3%EA%B8%B0_%EA%B3%BC%EC%A0%9C_(Catboostregressor_%ED%99%9C%EC%9A%A9)_public_6_0247.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor

import warnings
warnings.simplefilter('ignore')
warnings.filterwarnings('ignore')
df_train=pd.read_csv('./dacon_data/train.csv')
df_test=pd.read_csv('./dacon_data/test.csv')
df_train['송하인_grid']=df_train['송하인_격자공간고유번호'].astype(str).apply(lambda x: x[:3]).astype(int)
df_train['수하인_grid']=df_train['수하인_격자공간고유번호'].astype(str).apply(lambda x: x[:3]).astype(int)
df_test['송하인_grid']=df_test['송하인_격자공간고유번호'].astype(str).apply(lambda x: x[:3]).astype(int)
df_test['수하인_grid']=df_test['수하인_격자공간고유번호'].astype(str).apply(lambda x: x[:3]).astype(int)

FileNotFoundError: ignored

In [ ]:
df_train['송수_grid']=df_train['송하인_grid'].astype(str).str.cat(df_train['수하인_grid'].astype(str), sep=' - ')
df_test['송수_grid']=df_test['송하인_grid'].astype(str).str.cat(df_test['수하인_grid'].astype(str), sep=' - ')

In [ ]:
df_train[~df_train['송수_grid'].isin(df_test['송수_grid'].unique())].head()

,index,송하인_격자공간고유번호,수하인_격자공간고유번호,물품_카테고리,운송장_건수,송하인_grid,수하인_grid,송수_grid
14316,14316,4223000062013100,5011000435058300,수산,3,422,501,422 - 501
15188,15188,3120000085064100,5011000265100400,기타식품,3,312,501,312 - 501
20364,20364,4223000062013100,5011000314094200,수산,5,422,501,422 - 501
20806,20806,4221000028003400,5013000926057200,기타패션의류,3,422,501,422 - 501


In [ ]:
df_test[~df_test['송수_grid'].isin(df_train['송수_grid'].unique())] #즉, test에 대해 학습이 가능한 column을 만들었다.

,index,송하인_격자공간고유번호,수하인_격자공간고유번호,물품_카테고리,송하인_grid,수하인_grid,송수_grid


In [ ]:
df_train['거래목록']=df_train['송수_grid'].str.cat(df_train['물품_카테고리'], sep=' , ')
df_test['거래목록']=df_test['송수_grid'].str.cat(df_test['물품_카테고리'], sep=' , ')

In [ ]:
df_cat=df_train.copy() #학습용 dataset
df_cat.columns

NameError: ignored

In [ ]:
remove_cols=['index', '송하인_격자공간고유번호', '수하인_격자공간고유번호', '송수_grid', '거래목록']
df_cat=df_cat.drop(remove_cols, axis=1, inplace=False)
df_cat_test=df_test.copy()
df_cat_test=df_cat_test.drop(remove_cols, axis=1, inplace=False) #시험용 dataset

In [ ]:
from sklearn.model_selection import StratifiedKFold
from catboost import CatBoostRegressor
# K Fold 및 기본 데이터 준비 
x_train=df_cat.drop(['운송장_건수'], axis=1, inplace=False)
y_train=df_cat['운송장_건수']


folds = StratifiedKFold(n_splits = 5, shuffle = True, random_state = 511) 
oof_preds = np.zeros(df_cat.shape[0])

# catboost 이용 
print("using catboost") 
sub_preds_catboost = np.zeros(df_cat_test.shape[0])


using catboost


In [ ]:
for n_fold, (trn_idx, val_idx) in enumerate(folds.split(df_cat, df_cat['운송장_건수'])) : 
    trn_x, trn_y = x_train.iloc[trn_idx], y_train.iloc[trn_idx]
    val_x, val_y = x_train.iloc[val_idx], y_train.iloc[val_idx]
    #oof_preds_catboost = np.zeros(val_y.shape[0])
    cb_model = CatBoostRegressor(iterations=1000,
                                 learning_rate=0.1,
                                 depth=4,
                                 l2_leaf_reg=20,
                                 bootstrap_type='Bernoulli',
                                 subsample=0.6,
                                 eval_metric='RMSE',
                                 metric_period=50,
                                 od_type='Iter',
                                 od_wait=45,
                                 random_seed=17,
                                 allow_writing_files=False)
    cb_model.fit(trn_x, trn_y, eval_set=(val_x, val_y), cat_features=['물품_카테고리'], use_best_model=True, verbose=True)
    #oof_preds_catboost[val_idx] = cb_model.predict(val_x)
    sub_preds_catboost += cb_model.predict(df_cat_test) / folds.n_splits


0:	learn: 6.8188888	test: 6.4445104	best: 6.4445104 (0)	total: 69.7ms	remaining: 1m 9s
50:	learn: 6.6377486	test: 6.2908028	best: 6.2908028 (50)	total: 750ms	remaining: 13.9s
100:	learn: 6.5827872	test: 6.2656556	best: 6.2656556 (100)	total: 1.41s	remaining: 12.5s
150:	learn: 6.5286270	test: 6.2503498	best: 6.2474250 (130)	total: 2.04s	remaining: 11.5s
Stopped by overfitting detector  (45 iterations wait)

bestTest = 6.247425028
bestIteration = 130

Shrink model to first 131 iterations.
0:	learn: 6.1634591	test: 8.6910318	best: 8.6910318 (0)	total: 16.9ms	remaining: 16.9s


50:	learn: 6.0655535	test: 8.6137421	best: 8.6135907 (46)	total: 663ms	remaining: 12.3s
100:	learn: 6.0338218	test: 8.5658457	best: 8.5658457 (100)	total: 1.31s	remaining: 11.7s
150:	learn: 6.0096021	test: 8.5407344	best: 8.5404026 (147)	total: 1.93s	remaining: 10.9s
200:	learn: 5.9920960	test: 8.5328177	best: 8.5328177 (200)	total: 2.53s	remaining: 10.1s
250:	learn: 5.9782880	test: 8.5229635	best: 8.5226932 (247)	total: 3.15s	remaining: 9.38s
300:	learn: 5.9553654	test: 8.5018169	best: 8.5012469 (294)	total: 3.78s	remaining: 8.79s
350:	learn: 5.9447665	test: 8.4981920	best: 8.4975318 (347)	total: 4.4s	remaining: 8.14s
400:	learn: 5.9314137	test: 8.4828759	best: 8.4827643 (398)	total: 5.01s	remaining: 7.48s
450:	learn: 5.9217675	test: 8.4805462	best: 8.4769358 (407)	total: 5.61s	remaining: 6.83s
Stopped by overfitting detector  (45 iterations wait)

bestTest = 8.476935769
bestIteration = 407

Shrink model to first 408 iterations.
0:	learn: 6.8731578	test: 6.1923763	best: 6.1923763 (0)	

50:	learn: 6.6709165	test: 6.1014790	best: 6.0938107 (41)	total: 636ms	remaining: 11.8s
Stopped by overfitting detector  (45 iterations wait)

bestTest = 6.093810704
bestIteration = 41

Shrink model to first 42 iterations.
0:	learn: 7.0161055	test: 5.5231686	best: 5.5231686 (0)	total: 18.1ms	remaining: 18.1s


50:	learn: 6.8554820	test: 5.4682927	best: 5.4560427 (21)	total: 655ms	remaining: 12.2s
Stopped by overfitting detector  (45 iterations wait)

bestTest = 5.456042736
bestIteration = 21

Shrink model to first 22 iterations.
0:	learn: 6.8172307	test: 6.4430648	best: 6.4430648 (0)	total: 15.7ms	remaining: 15.7s


50:	learn: 6.6712555	test: 6.2119236	best: 6.2119236 (50)	total: 636ms	remaining: 11.8s
100:	learn: 6.6332200	test: 6.1615890	best: 6.1615890 (100)	total: 1.24s	remaining: 11s
150:	learn: 6.6134652	test: 6.1450442	best: 6.1450427 (149)	total: 1.84s	remaining: 10.4s
200:	learn: 6.5942311	test: 6.1338612	best: 6.1317126 (188)	total: 2.42s	remaining: 9.6s
250:	learn: 6.5825551	test: 6.1286331	best: 6.1277060 (240)	total: 3s	remaining: 8.94s
300:	learn: 6.5692339	test: 6.1251588	best: 6.1239598 (282)	total: 3.57s	remaining: 8.28s
350:	learn: 6.5578785	test: 6.1174506	best: 6.1172928 (348)	total: 4.15s	remaining: 7.68s
400:	learn: 6.5397831	test: 6.1154245	best: 6.1093978 (355)	total: 4.75s	remaining: 7.09s
Stopped by overfitting detector  (45 iterations wait)

bestTest = 6.109397849
bestIteration = 355

Shrink model to first 356 iterations.


In [ ]:
sub_preds_catboost

array([5.44373783, 5.65467802, 5.85152653, ..., 4.11285451, 4.25377107,
       5.21755963])

In [ ]:
submission=pd.read_csv('./dacon_data/sample_submission.csv')
submission.head()

,index,운송장_건수
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0


In [ ]:
submission['운송장_건수']=sub_preds_catboost
submission.head()

,index,운송장_건수
0,0,5.443738
1,1,5.654678
2,2,5.851527
3,3,5.454960
4,4,4.521766


In [ ]:
submission.to_csv('./dacon_data/catboost_regressor_1st.csv', index=False)